# Part 0

## Create a Strava API Application

Follow the steps in this article to create a Strava API application.

https://medium.com/@lejczak.learn/get-your-strava-activity-data-using-python-2023-%EF%B8%8F-b03b176965d0

Use this URL and replace [client_id] with your client id.

https://www.strava.com/oauth/authorize?client_id=[client_id]&response_type=code&redirect_uri=http://localhost/exchange_token&approval_prompt=force&scope=activity:read_all




Copy code from localhost URL and paste here. Then, copy the below script and paste it into Terminal / Command Prompt to receive refresh token.

curl -X POST https://www.strava.com/oauth/token \
-F client_id=[client_id] \
-F client_secret=[client_secret] \
-F code=[code] \
-F grant_type=authorization_code

Once we have our client id, client secret, and refresh token, we are ready to use Strava's API.

# Part 1

## Using Strava API to retrieve data

In [1]:
import numpy as np
import pandas as pd
import requests
import urllib3
import time
from calendar import timegm
from osc_gen import sig
from osc_gen import wavetable
from osc_gen import visualize

# disable warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Defining functions to scrape Strava and transform data

In [4]:
# defining function to compress/stretch data to be of length 2048, peak between -1 and 1, and have mean 0
def wav_transform(arr, numSamples=2048):

    transform = np.interp(np.linspace(0, len(arr), numSamples), np.arange(len(arr)), arr)
    transform -= np.mean(transform)
    transform /= np.max(abs(transform))
   
    return transform

Prior to executing the code below, I ran the loop_through_pages function once to retrieve all of my activities, and I stored the resulting output into a .csv file titled 'strava_activities.csv'

In [ ]:
# defining function to iterate through Strava API
def loop_through_pages(page):
    # start at page ...
    page = page
    # set new_results to True initially
    new_results = True
    # create an empty array to store our combined pages of data in
    data = []
    while new_results:
        # Give some feedback
        print(f'You are requesting page {page} of your activities data ...')
        # request a page + 200 results
        get_strava = requests.get(activities_url, headers=header, params={'per_page': 200, 'page': f'{page}'}).json()
        # save the response to new_results to check if its empty or not and close the loop
        new_results = get_strava
        # add our responses to the data array
        data.extend(get_strava)
        # increment the page
        page += 1
    # return the combine results of our get requests
    return data

loop_through_pages(1)

## Setting up variables to pass to API and pull new activities

In [5]:
# setting location to save .wav files
location = "/Users/kevinstabinski/Dropbox/YouTube/Turning Fitness Data Into Sound/data/Kev's Kits - Movements/wavs"

# read activities .csv and get time of most recent activity
activities = pd.read_csv('strava_activities.csv')
utc_time = time.strptime(activities['start_date'].max(), "%Y-%m-%dT%H:%M:%SZ")
epoch_time = timegm(utc_time) + 1


# parameters for API
client_id     = 'xxx'
client_secret = 'xxx'
refresh_token = 'xxx'

# create urls to get access token and activities
auth_url = 'https://www.strava.com/oauth/token'
activities_url = 'https://www.strava.com/api/v3/athlete/activities'

# information to recieve access token
payload = {
    'client_id': f'{client_id}',
    'client_secret': f'{client_secret}',
    'refresh_token': f'{refresh_token}',
    'grant_type': "refresh_token",
    'f': 'json'
}

# request to get access token and create header with access token
res = requests.post(auth_url, data = payload, verify = False)
access_token = res.json()['access_token']
header = {'Authorization': 'Bearer ' + access_token}

# pull all activities after most recent activity
get_strava = requests.get(activities_url, headers=header, params={'after': epoch_time}).json()

# save new activities as dataframe
new_activities = pd.json_normalize(get_strava)

In [6]:
# if there are new activities, add them to the activities .csv and save
if not new_activities.empty:
    activities = pd.concat([activities, new_activities], ignore_index= True)
    activities = activities.sort_values('start_date', ascending = False)
    
    
    activities.to_csv('strava_activities.csv', index = False)

In [8]:
# go through new activities from API pull
for i in range(len(new_activities)):

    
    # obtain necessary variables from activity
    id = new_activities['id'][i]
    start_time = new_activities['start_date'][i]
    date = start_time.split('T')[0]
    type = new_activities['type'][i].lower()

    # specify API url
    routes_url = f"https://www.strava.com/api/v3/activities/{id}/streams"

    # only perform transformation on activities that have positional data
    if type == 'run' or type == 'ride' or type == 'walk':
        # obtain latitude, longitude, elevation for each activity
        latlong = requests.get(routes_url, headers=header, params={'keys': ['latlng']}).json()[0]['data']
        elevation = requests.get(routes_url, headers=header, params={'keys': ['altitude']}).json()[1]['data']

        # create dataframe with this variables
        data = pd.DataFrame([*latlong], columns=['latitude','longitude'])
        data['elevation'] = elevation

        # perform transformation and store as another dataframe
        wav_data = {'longitude' : wav_transform(data['longitude']),
                    'latitude'  : wav_transform(data['latitude']),
                    'elevation' : wav_transform(data['elevation'])}
        wav_data = pd.DataFrame(data = wav_data)


        # create objects from osc_gen
        sg = sig.SigGen()
        wt = wavetable.WaveTable(16)


        # morph between longitude, elevation, latitude to give it a smooth feel
        wt.waves = sig.morph((wav_data['longitude'], wav_data['elevation'], wav_data['latitude'] ), 16)

        # name file and save as .wav
        wavname = location + '/' f'kev_{type}_{date}.wav'
        wt.to_wav(wavname, samplerate=48000)